# **Data Understanding & Data Cleaning**

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
url = 'https://drive.google.com/uc?id=1PkyPBg-ZeCzcqnmVImlEzJqbmIa9z8ah'
df = pd.read_csv(url, sep=';', encoding='latin-1')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01/12/2010 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01/12/2010 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01/12/2010 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01/12/2010 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01/12/2010 08:26,3.39,17850.0,United Kingdom


In [ ]:
df["Description"].nunique()

4223

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [ ]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,541909.0,9.552250,218.081158,-80995.00,1.00,3.00,10.00,80995.0
UnitPrice,541909.0,4.611114,96.759853,-11062.06,1.25,2.08,4.13,38970.0
CustomerID,406829.0,15287.690570,1713.600303,12346.00,13953.00,15152.00,16791.00,18287.0


## **Mengatasi Missing Value**

In [ ]:
df.isnull().sum()

,0
InvoiceNo,0
StockCode,0
Description,1454
Quantity,0
InvoiceDate,0
UnitPrice,0
CustomerID,135080
Country,0


In [ ]:
df = df.dropna()
df.isna().sum()

,0
InvoiceNo,0
StockCode,0
Description,0
Quantity,0
InvoiceDate,0
UnitPrice,0
CustomerID,0
Country,0


## **Mengatasi Data Duplikat**

In [ ]:
df.duplicated().sum()

np.int64(5225)

In [ ]:
df = df.drop_duplicates()
df.duplicated().sum()

np.int64(0)

## **Mengubah Tipe Data pada  ```InvoiceDate``` dan ```CustomerID```**



In [ ]:
df.dtypes

,0
InvoiceNo,object
StockCode,object
Description,object
Quantity,int64
InvoiceDate,object
UnitPrice,float64
CustomerID,float64
Country,object


In [ ]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], format='%d/%m/%Y %H:%M')
df['CustomerID'] = df['CustomerID'].astype(str)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 401604 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    401604 non-null  object        
 1   StockCode    401604 non-null  object        
 2   Description  401604 non-null  object        
 3   Quantity     401604 non-null  int64         
 4   InvoiceDate  401604 non-null  datetime64[ns]
 5   UnitPrice    401604 non-null  float64       
 6   CustomerID   401604 non-null  object        
 7   Country      401604 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 27.6+ MB


## **Mengatasi Outlier**

In [ ]:
numeric_columns = df.select_dtypes(include=['int64','float64']).columns

df_clean = df.copy()

for c in numeric_columns:
    Q1 = df_clean[c].quantile(0.25)
    Q3 = df_clean[c].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR

    df_clean = df_clean[(df_clean[c] >= lower) & (df_clean[c] <= upper)]

## **Menghapus Nilai Negatif**

In [ ]:
(df_clean['Quantity'] < 0).sum()

np.int64(6188)

In [ ]:
df_clean = df_clean[df_clean['Quantity'] > 0]
(df_clean['Quantity'] < 0).sum()

np.int64(0)

# **Feature Engineering**

## **Membuat Fitur Kategori**

In [ ]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=0ce4b90b25e07d769f4b3c00df949c0712c6f9028f08fc6212a464cc180094ee
  Stored in directory: /root/.cache/pip/wheels/c1/67/88/e844b5b022812e15a52e4eaa38a1e709e99f06f6639d7e3ba7
Successfully built langdetect


In [ ]:
!pip install Sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 8.3 MB/s eta 0:00:00


In [ ]:
import re
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download("stopwords")
nltk.download("wordnet")

stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    if pd.isna(text):
        return ""
    text = text.lower()

    # keep words, numbers, hyphens
    text = re.sub(r"[^a-z0-9\- ]", " ", text)  # keep A-Z, 0-9, dash
    text = re.sub(r'\s+', ' ', text).strip()

    return text

def preprocess_text(text):
    text = clean_text(text)
    tokens = text.split()

    tokens = [t for t in tokens if t not in stop_words]     # english stopwords
    tokens = [lemmatizer.lemmatize(t) for t in tokens]      # english lemmatization

    return " ".join(tokens)

df_clean["Description_baru"] = df_clean["Description"].apply(preprocess_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 91.4 MB/s eta 0:00:00


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from gensim.models import CoherenceModel
from gensim import corpora
import numpy as np

def nmf_optimal_topics(corpus, n_topic_range=range(5,16), n_top_words=10,
                       min_df=5, max_df=0.85, ngram_range=(1,2),
                       custom_stopwords=None, random_state=42,
                       coherence_type='c_v', max_iter=1000):

    # TF-IDF
    vectorizer = TfidfVectorizer(
        stop_words=custom_stopwords,
        min_df=min_df,
        max_df=max_df,
        ngram_range=ngram_range
    )
    X = vectorizer.fit_transform(corpus)
    feature_names = np.array(vectorizer.get_feature_names_out())

    texts = [doc.split() for doc in corpus]
    dictionary = corpora.Dictionary(texts)

    coherence_per_n_topics = []
    nmf_models = {}

    for n_topics in n_topic_range:
        nmf_model = NMF(
            n_components=n_topics,
            random_state=random_state,
            max_iter=max_iter
        )
        nmf_model.fit(X)

        # Ambil top 10 kata per topik
        topics_for_coherence = []
        for topic in nmf_model.components_:
            top_idx = topic.argsort()[-10:][::-1]
            top_words = [feature_names[i] for i in top_idx]
            topics_for_coherence.append(top_words)

        cm = CoherenceModel(
            topics=topics_for_coherence, texts=texts,
            dictionary=dictionary, coherence=coherence_type
        )
        score = cm.get_coherence()
        coherence_per_n_topics.append((n_topics, score))
        nmf_models[n_topics] = nmf_model
        print(f"n_topics={n_topics}, coherence={score:.4f}")

    optimal_n_topics, final_coherence = max(coherence_per_n_topics, key=lambda x: x[1])
    print(f"\nOptimal n_topics = {optimal_n_topics} dengan coherence = {final_coherence:.4f}")

    nmf_model = nmf_models[optimal_n_topics]

    top_words_per_topic = []
    for topic in nmf_model.components_:
        top_idx = topic.argsort()[-n_top_words:][::-1]
        top_words = [feature_names[i] for i in top_idx]
        top_words_per_topic.append(top_words)

    return top_words_per_topic, nmf_model, X, final_coherence, coherence_per_n_topics, optimal_n_topics

top_words, nmf_model, X, final_coherence, coherence_list, optimal_n = nmf_optimal_topics(
    df_clean['Description_baru'],
    n_topic_range=range(3,20),
    n_top_words=12,
    ngram_range=(1,3),
    min_df=4,
    max_df=0.75
)

for i, words in enumerate(top_words):
    print(f"Topic {i}: {', '.join(words)}")

In [ ]:
# Dapatkan distribusi topic per document
topic_distribution = nmf_model.transform(X)

# Ambil topic dengan nilai tertinggi
df_clean['Topic'] = topic_distribution.argmax(axis=1)

In [ ]:
topic_to_category = {
    0: "Vintage Jumbo Bags",
    1: "Hot Water Bottle",
    2: "Hanging Light Decor",
    3: "Retrospot Red Collection",
    4: "Lunch Bags",
    5: "Bakelite Alarm Clocks",
    6: "Kitchen Tins & Pantry Set",
    7: "Metal Signs",
    8: "Pink/Charlotte Bags",
    9: "Christmas Paper Craft Kits",
    10: "Wicker Heart Decor",
    11: "Party Buntings",
    12: "Cake Cases & Baking Items",
    13: "Regency Teacup & Saucer",
    14: "Dolly Girl (Lunch & Accesories)"
}
df_clean['Category'] = df_clean['Topic'].map(topic_to_category)

In [ ]:
df_clean = df_clean[['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country', 'Category', 'YearMonth']]

In [ ]:
df_clean.head()

## **Monetary**

In [ ]:
df = df_clean.copy()  # pastikan df_clean sudah ada
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['Revenue'] = df['Quantity'] * df['UnitPrice']
df['YearMonth'] = df['YearMonth'].astype(str)  # jaga konsistensi
# pastikan CustomerID numeric and no NaN
df['CustomerID'] = pd.to_numeric(df['CustomerID'], errors='coerce')
df = df.dropna(subset=['CustomerID']).copy()
df['CustomerID'] = df['CustomerID'].astype(int)

In [ ]:
spend = df.groupby('CustomerID').agg(
    total_spend = ('Revenue','sum'),
    total_transactions = ('InvoiceNo','nunique'),
    total_items = ('Quantity','sum')
).reset_index()

In [ ]:
# avg order value per invoice
invoice_rev = df.groupby(['CustomerID','InvoiceNo'])['Revenue'].sum().reset_index()
avg_order = invoice_rev.groupby('CustomerID')['Revenue'].mean().rename('avg_order_value').reset_index()
max_order = invoice_rev.groupby('CustomerID')['Revenue'].max().rename('max_order_value').reset_index()
std_order = invoice_rev.groupby('CustomerID')['Revenue'].std().fillna(0).rename('std_order_value').reset_index()

spend = spend.merge(avg_order, on='CustomerID', how='left')\
             .merge(max_order, on='CustomerID', how='left')\
             .merge(std_order, on='CustomerID', how='left')

# spend velocity: average monthly spend
monthly_spend = df.groupby(['CustomerID','YearMonth'])['Revenue'].sum().reset_index()
spend_monthly_mean = monthly_spend.groupby('CustomerID')['Revenue'].mean().rename('spend_velocity').reset_index()
spend = spend.merge(spend_monthly_mean, on='CustomerID', how='left').fillna(0)

In [ ]:
def compute_trend(x):
    # x is monthly_spend series per customer with YearMonth as index
    if len(x) < 2:
        return 0.0
    y = x.values
    # x axis: 0..n-1
    xs = np.arange(len(y))
    a = np.polyfit(xs, y, 1)[0]
    return float(a)

trend = monthly_spend.sort_values(['CustomerID','YearMonth']).groupby('CustomerID').apply(
    lambda g: compute_trend(g['Revenue'].reset_index(drop=True))
).rename('spend_trend').reset_index()
spend = spend.merge(trend, on='CustomerID', how='left').fillna(0)

df_spend = spend.copy()

In [ ]:
df_spend.head(10)

In [ ]:
df_spend.to_csv("spend.csv", index = False)

## **Frequency**

In [ ]:
freq = df.groupby('CustomerID').agg(
    purchase_frequency = ('InvoiceNo','nunique'),
    unique_visit_days = ('InvoiceDate', lambda x: x.dt.date.nunique())
).reset_index()

# mean transactions per month
monthly_tx = df.groupby(['CustomerID','YearMonth'])['InvoiceNo'].nunique().reset_index(name='txs')
freq_monthly_mean = monthly_tx.groupby('CustomerID')['txs'].mean().rename('purchase_per_month').reset_index()
freq = freq.merge(freq_monthly_mean, on='CustomerID', how='left')

# interpurchase intervals
def ipi_stats(dates):
    s = dates.sort_values()
    diffs = s.diff().dt.days.dropna()
    return pd.Series({
        'order_interval_mean': float(diffs.mean()) if len(diffs)>0 else 0.0,
        'order_interval_std': float(diffs.std()) if len(diffs)>1 else 0.0
    })

ipi = df.groupby('CustomerID')['InvoiceDate'].apply(lambda x: ipi_stats(x)).unstack().reset_index()
freq = freq.merge(ipi, on='CustomerID', how='left').fillna(0)

df_freq = freq.copy()

In [ ]:
df_freq.head(10)

In [ ]:
df_freq.to_csv("freq.csv", index = False)

## **Recency**

In [ ]:
last_date = df['InvoiceDate'].max()
recency = df.groupby('CustomerID')['InvoiceDate'].max().rename('last_purchase').reset_index()
recency['days_since_last_purchase'] = (last_date - recency['last_purchase']).dt.days

# time since first purchase
first = df.groupby('CustomerID')['InvoiceDate'].min().rename('first_purchase').reset_index()
recency = recency.merge(first, on='CustomerID')
recency['time_since_first_purchase'] = (last_date - recency['first_purchase']).dt.days

# average recency (mean gap)
avg_gap = df.groupby('CustomerID')['InvoiceDate'].apply(lambda x: x.sort_values().diff().dt.days.mean()).rename('avg_recency').reset_index()
recency = recency.merge(avg_gap, on='CustomerID', how='left').fillna(0)

# recency rank / zscore
recency['recency_rank'] = recency['days_since_last_purchase'].rank(method='dense', ascending=True).astype(int)
recency['recency_zscore'] = (recency['days_since_last_purchase'] - recency['days_since_last_purchase'].mean())/recency['days_since_last_purchase'].std()
recency['recency_zscore'] = recency['recency_zscore'].fillna(0)

df_recency = recency.copy()

In [ ]:
df_recency.head(10)

In [ ]:
df_recency.to_csv("recency.csv", index = False)

## **Category**

In [ ]:
# pivot counts and spend per category
cat_qty = df.pivot_table(index='CustomerID', columns='Category', values='Quantity', aggfunc='sum', fill_value=0)
cat_rev = df.pivot_table(index='CustomerID', columns='Category', values='Revenue', aggfunc='sum', fill_value=0)

# category share (percent)
cat_share = cat_qty.div(cat_qty.sum(axis=1).replace(0,1), axis=0).add_prefix('share_')

# dominant categories and diversity/entropy
dominant_1 = cat_share.idxmax(axis=1).rename('top_category_1').reset_index()
dominant_2 = cat_share.apply(lambda row: row.sort_values(ascending=False).index[1] if row.sum()>0 and len(row[row>0])>1 else np.nan, axis=1).rename('top_category_2').reset_index()

# category diversity = number of categories with >0 purchases
cat_diversity = (cat_qty>0).sum(axis=1).rename('category_diversity').reset_index()

# entropy
cat_entropy = (-(cat_share.replace(0,1e-9)*np.log(cat_share.replace(0,1e-9))).sum(axis=1)).rename('category_entropy').reset_index()

df_cat_feats = cat_share.reset_index().merge(dominant_1, on='CustomerID', how='left')\
                                    .merge(dominant_2, on='CustomerID', how='left')\
                                    .merge(cat_diversity, on='CustomerID', how='left')\
                                    .merge(cat_entropy, on='CustomerID', how='left')

df_category = df_cat_feats.copy()

In [ ]:
df_category.head(10)

In [ ]:
df_category.to_csv("category.csv", index = False)

## **Basket**

In [ ]:
invoice_items = df.groupby(['CustomerID','InvoiceNo']).agg(
    basket_qty = ('Quantity','sum'),
    basket_value = ('Revenue','sum'),
    unique_skus = ('StockCode', 'nunique')
).reset_index()

basket = invoice_items.groupby('CustomerID').agg(
    avg_basket_size = ('basket_qty','mean'),
    basket_std = ('basket_qty','std'),
    avg_basket_value = ('basket_value','mean'),
    basket_value_std = ('basket_value','std'),
    avg_unique_items = ('unique_skus','mean')
).fillna(0).reset_index()

df_basket = basket.copy()

In [ ]:
df_basket.head(10)

In [ ]:
df_basket.to_csv("basket.csv", index=False)

## **Time**

In [ ]:
df['hour'] = df['InvoiceDate'].dt.hour
df['dow'] = df['InvoiceDate'].dt.day_name()

time_feats = df.groupby('CustomerID').agg(
    morning_ratio = ('hour', lambda x: ((x>=6)&(x<12)).mean()),
    afternoon_ratio = ('hour', lambda x: ((x>=12)&(x<17)).mean()),
    evening_ratio = ('hour', lambda x: ((x>=17)&(x<24)).mean()),
    night_ratio = ('hour', lambda x: ((x>=0)&(x<6)).mean()),
    weekend_ratio = ('dow', lambda x: x.isin(['Saturday','Sunday']).mean())
).reset_index()

# peak hour (mode)
peak_hour = df.groupby('CustomerID')['hour'].agg(lambda x: x.mode().iloc[0] if len(x.mode())>0 else np.nan).rename('peak_hour').reset_index()
time_feats = time_feats.merge(peak_hour, on='CustomerID', how='left')

# seasonality: monthly purchase std/mean
monthly_counts = df.groupby(['CustomerID','YearMonth'])['InvoiceNo'].nunique().reset_index(name='txs')
season = monthly_counts.groupby('CustomerID').agg(
    monthly_txs_mean = ('txs','mean'),
    monthly_txs_std = ('txs','std')
).fillna(0).reset_index()

df_time = time_feats.merge(season, on='CustomerID', how='left')

In [ ]:
df_time.head()

In [ ]:
df_time.to_csv("time.csv", index=False)

## **Merge DataFrame RFM**

In [ ]:
dfs = [df_spend, df_freq, df_recency, df_category, df_basket, df_time, df_value]
from functools import reduce
master_features = reduce(lambda left,right: left.merge(right, on='CustomerID', how='outer'), dfs)

# Before filling NaNs, convert 'Categorical' columns that store numeric labels
# to a numeric type if '0' is intended as a fill value.
# The R/F/M quartile columns are likely the culprits.
import pandas as pd
for col in ['R_quartile', 'F_quartile', 'M_quartile']:
    if col in master_features.columns and isinstance(master_features[col].dtype, pd.CategoricalDtype):
        master_features[col] = master_features[col].astype(float) # Convert to float to allow NaNs and then fill with 0

master_features = master_features.fillna(0)

# optional: tidy columns order (CustomerID first)
cols = ['CustomerID'] + [c for c in master_features.columns if c!='CustomerID']
master_features = master_features[cols]

In [ ]:
master_features.head(10)